Goal: Determine whether substraction method applied to overall yield good number
1. Number are positives
2. Sum up to total

### 1. Fetching the data

In [ ]:
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt

def run_query(client: bigquery.Client, query: str):
    df = client.query(query).to_dataframe()
    return df

In [ ]:
client = bigquery.Client()

In [ ]:
organics_overall = """
--- substraction method for organics, regardless of platform

with double_counts as (
  select
    date_trunc(hau_date, isoweek) as isoweek,
    country_code,
    COUNTIF(utm_source in ('google', 'Apple') AND old_hau = 'tvstreaming') as double_count,
    countif(old_hau = 'tvstreaming') as attribution_count,
  --  from `relax-melodies-android.late_conversions.users_network_attribution`
  from `relax-melodies-android`.`late_conversions`.`users_network_attribution`
  where
    hau is not null
    and utm_source is not null
  group by
    date_trunc(hau_date, isoweek), country_code
), double_counting as (
  select
    isoweek,
    country_code,
    double_count,
    attribution_count,
    SAFE_DIVIDE(double_count, attribution_count) as double_counting_perc,
    -- case when attribution_count > 0
    --   then double_count / attribution_count
    --   else 0
    -- end as double_counting_perc,
  from double_counts
), af_aggregate as ( # FIXME?
  select
    date, country, 
    sum(installs) as installs,
    sum(trials) as trials,
    sum(paid) as paid,
    sum(revenues) as revenues
  from `relax-melodies-android`.`ua_dashboard_prod`.`pre_final_view`
  where
    network = 'Appsflyer Aggregate'
  group by date, country
), tatari_aggregate as (
  select
    date, country,
    sum(installs) as installs,
    sum(trials) as trials,
    sum(paid) as paid,
    sum(revenues) as revenues
  from `relax-melodies-android`.`ua_dashboard_prod`.`pre_final_view`
  where
    network in ('tatari_streaming', 'tatari_linear', 'tatari_programmatic')
  group by date, country
), accounted_networks as (
  select
    date, country,
    sum(installs) as installs,
    sum(trials) as trials,
    sum(paid) as paid,
    sum(revenues) as revenues
  from `relax-melodies-android`.`ua_dashboard_prod`.`pre_final_view`
  where
    network in ('Apple Search Ads', 'Facebook Ads', 'snapchat_int', 'tiktokglobal_int', 'googleadwords_int', 'tatari_linear', 'tatari_streaming', 'tatari_programmatic')
  group by date, country
), conversions_rates as (
  select 
    install_date, country
    , modeled_trial2paid as t2p
    , modeled_revenue_per_paid as rev_per_paid
    , modeled_paid2refund as paid2refund
    , modeled_refunded_amount_per_paid as refunded_amound_per_paid
  --  from `relax-melodies-android.ua_transform_prod.trial2paid_ensemble`
  from `relax-melodies-android`.`ua_transform_prod`.`trial2paid_ensemble`
  where 
    network = 'Organic'
), organic_estimation as (
  select
    af.date,
    'Organic' as network,
    af.country,
    -- Formula: Appsflyer Aggregate - (Total Attributed Paid - Double Counting % * Tatari)
    -- Which simplifies to: AF - Total Paid + (Double Counting % * Tatari)
    COALESCE(af.installs, 0) - COALESCE(n.installs, 0) + COALESCE(dc.double_counting_perc, 0) * COALESCE(t.installs, 0) as installs,
    COALESCE(af.trials, 0) - COALESCE(n.trials, 0) + COALESCE(dc.double_counting_perc) * COALESCE(t.trials, 0) as trials,
    COALESCE(af.paid, 0) - COALESCE(n.paid, 0) + COALESCE(dc.double_counting_perc) * COALESCE(t.paid, 0) as paid,
    COALESCE(af.revenues, 0) - COALESCE(n.revenues, 0) + COALESCE(dc.double_counting_perc) * COALESCE(t.revenues, 0) as revenues,
    dc.double_counting_perc as double_counting_perc,
  from af_aggregate af
  left join tatari_aggregate t
    on af.date = t.date
      and af.country = t.country
  left join double_counting dc
    on date_trunc(af.date, isoweek) = dc.isoweek
      and af.country = dc.country_code
  left join accounted_networks n
    on af.date = n.date
    and af.country = n.country
), organic_estimation_clean as (
  select 
    o.date, o.network, o.country, 
    o.installs, 
    --- TRIALS
    case 
      -- (o.trials/o.paid) >= 0.8
      when (o.paid > 0 and c.t2p > 0.1 and (o.paid / c.t2p <= o.installs)) or (o.trials < 0.0) then safe_divide(o.paid, c.t2p)
      else o.trials
    end as trials,
    -- c.t2p as t2p, 
    -- c.rev_per_paid,
    --- PAID
    o.paid,
    --- REVENUE
    case 
      when o.paid <= 0 then 0.0
      when (o.revenues >= o.paid * 60) or (o.revenues = 0.0 and o.paid > 0.0) then o.paid * c.rev_per_paid
      else o.revenues
    end as revenues,
    double_counting_perc,
  from organic_estimation o 
  left join conversions_rates c
    on o.date = c.install_date
    and o.country = c.country
)

select * from organic_estimation_clean
"""

In [ ]:
df = run_query(client, organics_overall)

In [ ]:
df.head()

### 2. EDA

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df_new = df[df['date'] > '2025-01-01']
df_new = df_new.sort_values(by=['date', 'country'])
df_new = df_new.fillna(0.0)

In [ ]:
def plot(df, country: str, metric: str, method: str = None):
    dff = df.loc[df['country'] == country, ['date', metric]]
    dff = dff.sort_values(by=['date'])
    title = f"Organics {metric} over time in {country} ({method})" if method else f"Organics {metric} over time in {country}"
    
    if method == 'rolling': 
        dff[metric] = dff[metric].rolling(window=7).mean()
        plt.plot(dff['date'], dff[metric])
    elif method == 'resample':
        dff.index = pd.to_datetime(dff['date'])
        dff[[metric]].resample("W").median().plot()
    elif method == 'diff':
        dff.index = pd.to_datetime(dff['date'])
        dff[[metric]].resample("W").median().diff().plot()
    else:
        plt.plot(dff['date'], dff[metric])
        
    plt.title(title)
    plt.show()

In [ ]:
# plot without smoothing
plot(df=df_new, country='US', metric='installs', method=None)
plot(df=df_new, country='US', metric='trials', method=None)
plot(df=df_new, country='US', metric='paid', method=None)
plot(df=df_new, country='US', metric='revenues', method=None)

In [ ]:
# --- plot with smoothing
plot(df=df_new, country='US', metric='installs', method='rolling')
plot(df=df_new, country='US', metric='trials', method='rolling')
plot(df=df_new, country='US', metric='paid', method='rolling')
plot(df=df_new, country='US', metric='revenues', method='rolling')

In [ ]:
# --- plot with smoothing
plot(df=df_new, country='US', metric='installs', method='resample')
plot(df=df_new, country='US', metric='trials', method='resample')
plot(df=df_new, country='US', metric='paid', method='resample')
plot(df=df_new, country='US', metric='revenues', method='resample')

In [ ]:
# --- plot with diff
plot(df=df_new, country='US', metric='installs', method='diff')
plot(df=df_new, country='US', metric='trials', method='diff')
plot(df=df_new, country='US', metric='paid', method='diff')
plot(df=df_new, country='US', metric='revenues', method='diff')